In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import helper

import json
import os
import sys


from selenium.webdriver.chrome.service import Service

In [2]:
# course_schedule = []
NO_WINDOW = True
IMPLICIT_WAIT_TIME = 15
EXPLICIT_WAIT_TIME = 15
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440

WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

# RETRY_INTERVAL = 65

# SIGIN_IN_EVERYTIME = True

In [3]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, wait, xpath):

    try:
        wait.until(ec.presence_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element("xpath",xpath)

def sync_get_elements_by_xpath(driver, wait, xpath):

    try:
        wait.until(ec.presence_of_all_elements_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_elements("xpath",xpath)

In [4]:
def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [5]:
def create_new_driver():
    #setup option for chrome profile
    chrome_options = Options()
    
    ## NEVER USE THE FOLLOWING OPTION SINCE IT WILL MAKE SUPPOSEDLY DIFFERENT WINDOWS FOR DIFFERENT CLASS MIXED IN SOME WAY!!!!
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    ## the following option is to solve the error if using the above cause a problem
    # chrome_options.add_argument("--remote-debugging-port=9222") 
    
    prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
    chrome_options.add_experimental_option("prefs", prefs)

    if NO_WINDOW:
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-gpu')
        print("LOG: NO WINDOW")
    
    #start web driver
    # driver = webdriver.Chrome(options=chrome_options)
    driver = webdriver.Chrome(options=chrome_options)

    driver.implicitly_wait(IMPLICIT_WAIT_TIME)
    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    
    return driver, wait


In [6]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    global idx
    
    username = sync_get_element_by_xpath(driver, wait, "//input[@id='ctl00_cphContents_txtUsername']")
    password = sync_get_element_by_xpath(driver, wait, "//input[@id='ctl00_cphContents_txtPassword']")

    # my account:

    # if username is not None:
    #     send_keys(username, "DN23115986")
    # if password is not None:
    #     send_keys(password, "YUX8EVHC")

    # # Fanny's account:
      
    if username is not None:
        send_keys(username, "DN23035242")
    if password is not None:
        send_keys(password, "NTW3BSFH")

    
    try:
        wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))
    except TimeoutException:
        print("Get element with class name of submit-button imeout")
        return None
    
    sign_in_btn = sync_get_element_by_xpath(driver, wait, "//div[@class='submit-button']")
    
    sign_in_btn.click()
    

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, wait, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), '預定課')]")
#     print("click_btn: ", click_btn)
    click_btn.click()
    
def click_book_this_class_now(driver, wait):

    click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), '現在就預訂此課')]")

    # click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), '現在就預定此課')]")

    if click_btn:
        click_btn.click()
        return click_btn


In [7]:
def getBookingStatusHTML(driver, wait):

    divs = sync_get_elements_by_xpath(driver, wait, "//div[@class='class-info']//div[@class='right ar']")
    if divs is None:
        return "cannot find status text in the current site"
    
    html = ""
    for div in divs:
        html = div.get_attribute("innerHTML")
        if "位子" not in html:
            # htmls = html.split("\n")
            # for html in htmls:
            #     if html.isspace():
            #         continue
            #     if len(html) == 0:
            #         continue
            #     if "input" in html:
            #         continue
            #     html = html.strip()
            break
    return html


In [8]:
def log_and_print(logString):
      global csv_file_name
      logString += "\n"
      print(logString)
      
      with open(csv_file_name, "a") as write_file:
          write_file.write(logString)

In [9]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [10]:
temp = []

# load schedule from file
##---------------------------------------------------------------

with open("./class_schedules/09 九月 - 15 九月 2024.json", "r") as read_file:
    course_schedule = json.load(read_file)


# -------------------------------------------------------

# Day_string = "09"
# ## Mon

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('Gen', c)
# # c = helper.search_courses_by_teacher('Una', c)

# # temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Uni', c)
# # c = helper.search_courses_by_teacher('Una', c)

# temp.extend(c)
# -------------------------------------------------------
# Day_string = "10"
# ## Tue

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('War', c)
# # c = helper.search_courses_by_teacher('Mei', c)

# # temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('H', c)
# c = helper.search_courses_by_teacher('Una', c)

# temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Intro', c)
# # c = helper.search_courses_by_teacher('Una', c)

# # temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Counter Flow 1', c)
# c = helper.search_courses_by_teacher('Marcus', c)

# temp.extend(c)
# -------------------------------------------------------
# Day_string = "17"
# ## Wed

# c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Una', c)

# temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Warm', c)
# # c = helper.search_courses_by_teacher('El', c)

# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Gen', c)
# # c = helper.search_courses_by_teacher('El', c)

# # temp.extend(c)


# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('入', c)
# # c = helper.search_courses_by_teacher('Joe', c)

# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Wall', c)
# # c = helper.search_courses_by_teacher('', c)

# # temp.extend(c)
# -------------------------------------------------------
# Day_string = "25"
# ## Thu

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Tone', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('', c)

# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Yin', c)
# c = helper.search_courses_by_teacher('Mei', c)
# # c = helper.search_courses_by_time('', c)

# temp.extend(c)
# -------------------------------------------------------
# Day_string = "19"
# # ## Fri

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('He', c)
# c = helper.search_courses_by_teacher('Tu', c)
# # c = helper.search_courses_by_time('', c)

# temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('Tone', c)
# # c = helper.search_courses_by_teacher('Una', c)
# # # c = helper.search_courses_by_time('', c)

# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Iyen', c)
# # c = helper.search_courses_by_teacher('Char', c)
# # # c = helper.search_courses_by_time('', c)

# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Wall', c)
# # c = helper.search_courses_by_teacher('Moor', c)
# # # c = helper.search_courses_by_time('', c)

# # temp.extend(c)
# -------------------------------------------------------
Day_string = "14"
## Sat

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Fl', c)
# c = helper.search_courses_by_teacher('Moor', c)
# # c = helper.search_courses_by_time('', c)

# temp.extend(c)

c = helper.search_courses_by_date(Day_string, course_schedule)
c = helper.search_courses_by_name('Yin', c)
c = helper.search_courses_by_teacher('W', c)
# c = helper.search_courses_by_time('', c)

temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Fl', c)
# c = helper.search_courses_by_teacher('Ja', c)
# # c = helper.search_courses_by_time('', c)

# temp.extend(c)
# -------------------------------------------------------
# Day_string = "08"   
# ## Sun

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Hatha', c)
# # c = helper.search_courses_by_teacher('Ya', c)
# # # c = helper.search_courses_by_time('', c)

# # temp.extend(c)
            
# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Universal Basic 寰宇瑜珈基礎', c)
# # c = helper.search_courses_by_teacher('Ya', c)
# # # c = helper.search_courses_by_time('', c)

# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Universal Yoga 寰宇瑜珈', c)
# # c = helper.search_courses_by_teacher('Ya', c)
# # # c = helper.search_courses_by_time('', c)

# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Yin', c)
# # c = helper.search_courses_by_teacher('Ya', c)
# # # c = helper.search_courses_by_time('', c)

# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Gen', c)
# # c = helper.search_courses_by_teacher('Una', c)
# # # c = helper.search_courses_by_time('', c)

# # temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Vinyasa Yoga 流暢瑜珈', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('', c)

# temp.extend(c)
##---------------------------------------------------------------

# c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Marcus', c)
# # c = helper.search_courses_by_time('', c)

# temp.extend(c)

# -------------------------------------------------------


courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 14 9月 周六
-----------------------

Name: Yin Yang Yoga 陰陽瑜珈
Teacher: William L.
Time: 12:30 - 01:30 下午




In [11]:
master_counter = 0

In [12]:
def sign_in_once(driver, wait):

    try:
        driver.get(base_url)
    except:
        pass

    sign_in(driver, wait)

In [13]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [14]:
csv_file_name = "./log_Fanny_dup1_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [15]:
def reserve_class(driver, wait, course_to_be_booked):
    
    global master_counter
    global booked_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    
    try:
        driver.get(book_url)
    except:
        pass
    master_counter = master_counter + 1
    
    statusHTML = getBookingStatusHTML(driver, wait)
    print("status html: " + statusHTML)
    log_str = "\n************************\nstatus html: " + statusHTML+"\n************************\n\n"


    if "您已預訂此課" in statusHTML:
        if course_to_be_booked not in booked_classes:
            booked_classes.append(course_to_be_booked)
        log_str += "{}: {} by {} was BOOKED before {}\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        
        return False
    if "此課程已無空缺！" in statusHTML:
        log_str += "{}: {} by {} is FULL at {}\n\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)

        return False
    
    if "現在就預訂此課" in statusHTML:
        if click_book_this_class_now(driver, wait):
            if course_to_be_booked not in booked_classes:
                booked_classes.append(course_to_be_booked)
        
            log_str += "{}: {} by {} is BOOKED at {}\n".format(master_counter, class_name, teacher, datetime.datetime.now())
            print(log_str)
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
                
            return True
    if "預訂開始日期" in statusHTML:
        booking_available_time = statusHTML.split("<br>")[1]
        log_str += "{}: {} by {} is not available for booking yet at {}. You should try after {}\n".format(master_counter, class_name, teacher, datetime.datetime.now(), booking_available_time)
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)

        return False
    if "線上預訂已截止" in statusHTML:
        if course_to_be_booked not in booked_classes:
            booked_classes.append(course_to_be_booked)
        log_str += "{}: Booking of \"{} by {}\" was CLOSED\n".format(master_counter, class_name, teacher)
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        
        return False
    
    if "我們檢測到來自您的地址的異常流量" in driver.page_source:

        log_str += "我們檢測到來自您的地址的異常流量\n"
        print(log_str)
        
        os.system("say {}".format("We have detected unusual traffic from your address"))

        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        
        return False

    if "We've detected unusual traffic from your address" in driver.page_source:

        log_str += "We've detected unusual traffic from your address\n"
        print(log_str)
        
        os.system("say {}".format("We have detected unusual traffic from your address"))

        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        
        return False
    
    
    with open(csv_file_name, "a") as write_file:
        log_str += "UNEXPECTED SCENARIO HAPPENED. NEED TO REVISE THE CODE\n\n Current Page Source:\n"
        log_str += driver.page_source
        print(log_str)
        
        os.system("say {}".format("Yoga Edition Class Booking. Network Error"))
        
        write_file.write(log_str)
    

    return False


In [ ]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

booked_classes = []
while True:

    # create all drivers

    for idx, course_to_be_booked in enumerate(courses_to_be_booked):
            if courses_to_be_booked[idx] in booked_classes:
                  print("skip ", courses_to_be_booked[idx])
                  drivers.append(None)
                  waits.append(None)
                  continue
            driver, wait = create_new_driver()
            log_and_print("create driver: [{}] at {}".format(driver,datetime.datetime.now()))
            
            driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
            driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
            
            drivers.append(driver)
            waits.append(wait)
    time.sleep(1)

    # sign in for all classes
    for idx, course_to_be_booked in enumerate(courses_to_be_booked):
        if courses_to_be_booked[idx] in booked_classes:
                  continue
        driver = drivers[idx]
        wait = waits[idx]
        
        sign_in_once(driver, wait)
        log_and_print("sign in for class idx:{} at {}".format(idx,datetime.datetime.now()))
        time.sleep(3)
    

    # reserve all classes 
    
    for idx, course_to_be_booked in enumerate(courses_to_be_booked):
            if course_to_be_booked in booked_classes:
                  continue
            driver = drivers[idx]
            wait = waits[idx]

            if reserve_class(driver, wait, course_to_be_booked):
                log_and_print("Successfully reserved the class idx:{} at {}".format(idx,datetime.datetime.now()))
                if course_to_be_booked not in booked_classes:
                    booked_classes.append(course_to_be_booked)
            
            else:
                log_and_print("Failed to reserve the class idx:{} at {}".format(idx,datetime.datetime.now()))
                      
    time.sleep(5)

    # close all drivers
    for driver in drivers:
      if driver:
        driver.close()
        log_and_print("close driver: [{}] at {}".format(driver,datetime.datetime.now()))

    drivers = []
    waits = []

    log_and_print("------------------\ncourses to be booked:\n")
    for course_to_be_booked in courses_to_be_booked:
        log_and_print(course_to_be_booked["name"])
    log_and_print("\n------------------\n")

    log_and_print("------------------\nbooked classes:\n")
    for booked_class in booked_classes:
        log_and_print(booked_class["name"])
    log_and_print("\n------------------\n")

    if len(booked_classes) == len(courses_to_be_booked):
          log_and_print("finish all {} classes booking".format(len(booked_classes)))
          break
    time.sleep(1)

LOG: NO WINDOW
create driver: [<selenium.webdriver.chrome.webdriver.WebDriver (session="6c0b9eb575a7253a5d3189703d87f57c")>] at 2024-09-11 22:10:44.766077

sign in for class idx:0 at 2024-09-11 22:10:51.078239

status html: 
                <input type="submit" name="ctl00$cphContents$btnBookSubmiter" value="" id="ctl00_cphContents_btnBookSubmiter" tabindex="-1" class="hidden">
                
                <input type="submit" name="ctl00$cphContents$btnWaitSubmiter" value="" id="ctl00_cphContents_btnWaitSubmiter" tabindex="-1" class="hidden">
                
                此課程已無空缺！
            

************************
status html: 
                <input type="submit" name="ctl00$cphContents$btnBookSubmiter" value="" id="ctl00_cphContents_btnBookSubmiter" tabindex="-1" class="hidden">
                
                <input type="submit" name="ctl00$cphContents$btnWaitSubmiter" value="" id="ctl00_cphContents_btnWaitSubmiter" tabindex="-1" class="hidden">
                
     